In [ ]:
# For COLAB
# !pip install torch
# !pip install torchvision
# !pip install pillow
# !pip install transformers
# !pip install tqdm

# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# train script
from tqdm import tqdm
from time import time
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from PIL import Image
from transformers import DeiTConfig, DeiTModel # CANNOT INCLUDE THIS IN FINAL SUBMISSION - REMOVE IT BEFORE SUBMITTING
import multiprocessing  # CANNOT INCLUDE THIS IN FINAL SUBMISSION - REMOVE IT BEFORE SUBMITTING


def set_device():
    """
    Set device: to either Cuda (GPU), MPS (Apple Silicon GPU), or CPU
    """
    dev = torch.device(
        'cuda'
        if torch.cuda.is_available()
        else 'mps'
        if torch.backends.mps.is_available()
        else 'cpu'
    )
    print(f'Using {dev} device')
    return dev
device = set_device()
start = time()
# # ViT: transform cifar-10 dataset for 'facebook/deit-...-patch16-224'
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# ViT: Apply transformation to CIFAR10 dataset (50,000 images):
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
print(f'Time to transformed images {round(time() - start, 4)} secs')
start = time()
batch_size = 20
# cpu_count_mp = multiprocessing.cpu_count()  # REMOVE BEFORE SUBMITTING
print(f"Number of CPU threads according to multiprocessing: {cpu_count_mp}") # REMOVE BEFORE SUBMITTING

# Load in Dataloader:  #  REMOVE BEFORE SUBMITING
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=cpu_count_mp)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

# The ground-truth `labels`, are recorded as integer from 0 to 9.
# The number corresponds to the string labels listed in this `classes` tuple.
# Hence, if the label at some position in trainset, say at `j`, i.e. labels[j], is 0, this indicates the true label
# for images[0] is 'plane':
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Create iterator of images:
dataiter = iter(trainloader)
print(f'Time to create iterator of dataloader images {round(time() - start, 4)} secs')
images, labels = next(dataiter)
print(f'images.shape={images.shape}')
# Save example images to jpg. These are not just random examples though? These are the batch of 20 images taken from the trainloader of training images.
im = Image.fromarray((torch.cat(images.split(1, 0), 3).squeeze() / 2 * 255 + .5 * 255).permute(1, 2, 0).numpy().astype('uint8'))
im.save('train_pt_images_vit.jpg')  # useful for checking with ground truth labels below (i.e. sanity check)
print('train_pt_images_vit.jpg saved.')
print('Ground truth labels:' + ' '.join('%5s' % classes[labels[j]] for j in range(batch_size)))

print(f'Time to load images to Dataloader {round(time() - start, 4)} secs')
# # CNN: instantiate untrained model
# net = Net()

Using cpu device
Files already downloaded and verified
Time to transformed images 2.6293 secs


NameError: name 'cpu_count_mp' is not defined

In [ ]:
print(f'images.shape={images.shape}')

In [28]:
start = time()

# # ViT: instantiate untrained model
# # Init DeiT `deit-tiny-patch16-224` style configuration, (with random weights)
config = DeiTConfig.from_pretrained('facebook/deit-tiny-patch16-224')
# net = DeiTModel(config)
# # print(f'Tiny net {net}')
#
# # config = DeiTConfig()  # empty constructor appears to be base model
net = DeiTModel(config)
# print(f'Default base net {net}')
#
net = net.to(device)

# with open('network_pt_vit.py', 'w') as f:  # Save empty model architecture to file
#     f.write(str(net))

# print(f'Time to initialise DeiT config and model {round(time() - start, 4)} secs')

You are using a model of type vit to instantiate a model of type deit. This is not supported for all configurations of models and can yield errors.


'/Users/beanburger/Documents/_ML_UCL/_ML_MODULES/T2_MODULES/ASSIGNMENTS2/ADL_CW/cw1_pt/task2/img_cls_to_vit'

In [29]:
torch.save(net, 'saved_models/deit_tiny_vit_before.pt')  # save starting state of model hence "before"

In [4]:
net = torch.load(f='saved_models/pretrained/deit_tiny_vit.pt')
net = net.to(device)

In [5]:
# print(net.type)

In [9]:
import numpy
dummy_img = numpy.random.randn(1,3,224, 224)
dummy_img = torch.from_numpy(dummy_img)

In [ ]:
def patchify(images, n_patches):
    n, c, h, w = images.shape

    assert h == w, "Patchify method is implemented for square images only"

    patches = torch.zeros(n, n_patches ** 2, h * w * c // n_patches ** 2)
    patch_size = h // n_patches

    for idx, image in enumerate(images):
        for i in range(n_patches):
            for j in range(n_patches):
                patch = image[:, i * patch_size: (i + 1) * patch_size, j * patch_size: (j + 1) * patch_size]
                patches[idx, i * n_patches + j] = patch.flatten()
    return patches

In [10]:
output = net(dummy_img)

In [11]:
output[:2][1].shape

torch.Size([1, 192])

In [12]:
type(output)

transformers.modeling_outputs.BaseModelOutputWithPooling

In [3]:
start = time()
# loss and optimiser
criterion = torch.nn.CrossEntropyLoss()
criterion = criterion.to(device)
optimiser = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

print(f'Time to initialise ce loss and sgd optimiser {round(time() - start, 4)} secs')

Time to initialise ce loss and sgd optimiser 0.0067 secs


In [ ]:
# # Loading model and optimizer state
# checkpoint = torch.load('checkpoint.pth')
# net.load_state_dict(checkpoint['model_state_dict'])
# optimiser.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
# loss = checkpoint['loss']

In [4]:
# epochs = 2  # only 2 used for CNN
epochs = 20  # 20 for ViT

# training loop
for epoch in tqdm(range(epochs)):  # loop over the dataset multiple times

    loss, running_loss = 0.0, 0.0
    # for i, data in enumerate(trainloader, 0):
    #         # data is list of [inputs, labels]
    #         inputs, labels = data
    print(f'\nEPOCH {epoch}')
    for i, data in enumerate(trainloader, 0):
        # print(f'trainloader data number {i}')
        inputs, labels = data[0].to(device), data[1].to(device)
        # zero the parameter gradients
        optimiser.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs)
        print(outputs)
        outputs = outputs.pooler_output
        loss = criterion(outputs, labels)
        loss.backward()
        optimiser.step()
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:  # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

    # Saving model state at each epoch
    torch.save({
                'epoch': epoch,
                'model_state_dict': net.state_dict(),
                'optimizer_state_dict': optimiser.state_dict(),
                'loss': loss,
                }, 'checkpoint.pth')
    print(f'loss={loss.item()} at epoch={epoch}')

print(f'Completed training for {epochs} epochs.')
print(f'Time taken to train model for {epochs} epochs = {round(((time() - start) / 60), 4)} mins')

torch.save(net.state_dict(), 'saved_model_vit.pt')  # save trained model

print('Trained model saved.')

  0%|          | 0/20 [00:00<?, ?it/s]


EPOCH 0


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x1097cc860>
Traceback (most recent call last):
  File "/Users/beanburger/miniconda3/envs/comp0197-cw1-pt/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/Users/beanburger/miniconda3/envs/comp0197-cw1-pt/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1443, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/Users/beanburger/miniconda3/envs/comp0197-cw1-pt/lib/python3.12/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/beanburger/miniconda3/envs/comp0197-cw1-pt/lib/python3.12/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/beanburger/miniconda3/envs/comp0197-cw1-pt/lib/python3.12/multiprocessing/connection.py",

KeyboardInterrupt: 

Mounted at /content/drive
